In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist, mnist, cifar10
from tensorflow.keras.models import Model

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [37]:
(fmnist_train, _), (fmnist_test, _) = fashion_mnist.load_data()

fmnist_train = fmnist_train.astype('float32') / 255.
fmnist_test = fmnist_test.astype('float32') / 255.

fmnist_train = tf.reshape(fmnist_train, [-1, 28, 28, 1])
fmnist_test = tf.reshape(fmnist_test, [-1, 28, 28, 1])

print (fmnist_train.shape)
print (fmnist_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [36]:
(mnist_train, _), (mnist_test, _) = mnist.load_data()

mnist_train = mnist_train.astype('float32') / 255.
mnist_test = mnist_test.astype('float32') / 255.

mnist_train = tf.reshape(mnist_train, [-1, 28, 28, 1])
mnist_test = tf.reshape(mnist_test, [-1, 28, 28, 1])

print (mnist_train.shape)
print (mnist_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [42]:
# Load the CIFAR-10 dataset
(cifar_train, _), (cifar_test, _) = cifar10.load_data()

# Resize and convert to grayscale
new_size = (28, 28)
cifar_train_resized = np.zeros((cifar_train.shape[0], new_size[0], new_size[1], 1), dtype=np.uint8)
cifar_test_resized = np.zeros((cifar_test.shape[0], new_size[0], new_size[1], 1), dtype=np.uint8)

for i in range(cifar_train.shape[0]):
    resized_image = cv2.resize(cifar_train[i], new_size, interpolation=cv2.INTER_LINEAR)
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2GRAY)
    cifar_train_resized[i, :, :, 0] = grayscale_image

for i in range(cifar_test.shape[0]):
    resized_image = cv2.resize(cifar_test[i], new_size, interpolation=cv2.INTER_LINEAR)
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2GRAY)
    cifar_test_resized[i, :, :, 0] = grayscale_image

cifar_train = cifar_train_resized
cifar_test = cifar_test_resized

cifar_train = cifar_train.astype('float32') / 255.
cifar_test = cifar_test.astype('float32') / 255.

# x_train_resized now contains CIFAR-10 images in 28x28x1 format
# You can use x_test_resized similarly for the test set
print(cifar_train.shape)
print(cifar_test.shape)

(50000, 28, 28, 1)
(10000, 28, 28, 1)


In [106]:
class ae(Model):
  def __init__(self):
    super(ae, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(32, 32, 1)),
      layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
  
autoencoder = ae()

autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [108]:
autoencoder.fit(cifar_train, cifar_train,
                epochs=10,
                shuffle=True,
                validation_data=(cifar_test, cifar_test))

Epoch 1/10
1875/1875 [==============================] - 9s 4ms/step - loss: 0.0074 - val_loss: 0.0024
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0020 - val_loss: 0.0018
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0016 - val_loss: 0.0015
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0012 - val_loss: 0.0014
Epoch 8/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 9/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 10/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.001

In [109]:
reconstructions = autoencoder.predict(cifar_train)

#print(mnist_train.shape, reconstructions.shape)
cifar_train_loss = tf.keras.losses.mae(cifar_train, reconstructions)
print(cifar_train_loss.shape)


1875/1875 [==============================] - 3s 1ms/step
(60000, 32, 32)


In [110]:
threshold = np.mean(cifar_train_loss) + np.std(cifar_train_loss)
print("Threshold: ", threshold)

def predict(model, data, threshold):
  reconstructions = model(data)
  loss = tf.keras.losses.mae(reconstructions, data)
  return tf.math.less(tf.reduce_mean(loss, axis=[1,2]), threshold)

def compute_accuracy(predictions):
    total_predictions = len(predictions)
    correct_predictions = np.sum(predictions)
    
    accuracy = correct_predictions / total_predictions
    return accuracy

Threshold:  0.04502763


In [99]:
# Load the MNIST dataset
(mnist_train, _), (mnist_test, _) = mnist.load_data()

# Resize images from 28x28 to 32x32 while keeping 1 channel
new_size = (32, 32)
x_train_resized = np.zeros((mnist_train.shape[0], new_size[0], new_size[1], 1), dtype=np.uint8)
x_test_resized = np.zeros((mnist_test.shape[0], new_size[0], new_size[1], 1), dtype=np.uint8)

for i in range(mnist_train.shape[0]):
    resized_image = cv2.resize(mnist_train[i], new_size, interpolation=cv2.INTER_LINEAR)
    x_train_resized[i, :, :, 0] = resized_image

for i in range(mnist_test.shape[0]):
    resized_image = cv2.resize(mnist_test[i], new_size, interpolation=cv2.INTER_LINEAR)
    x_test_resized[i, :, :, 0] = resized_image

mnist_train = x_train_resized
mnist_test = x_test_resized

mnist_train = mnist_train.astype('float32') / 255.
mnist_test = mnist_test.astype('float32') / 255.

In [100]:
# Load the MNIST dataset
(fmnist_train, _), (fmnist_test, _) = fashion_mnist.load_data()

# Resize images from 28x28 to 32x32 while keeping 1 channel
new_size = (32, 32)
x_train_resized = np.zeros((fmnist_train.shape[0], new_size[0], new_size[1], 1), dtype=np.uint8)
x_test_resized = np.zeros((fmnist_test.shape[0], new_size[0], new_size[1], 1), dtype=np.uint8)

for i in range(fmnist_train.shape[0]):
    resized_image = cv2.resize(fmnist_train[i], new_size, interpolation=cv2.INTER_LINEAR)
    x_train_resized[i, :, :, 0] = resized_image

for i in range(fmnist_test.shape[0]):
    resized_image = cv2.resize(fmnist_test[i], new_size, interpolation=cv2.INTER_LINEAR)
    x_test_resized[i, :, :, 0] = resized_image

fmnist_train = x_train_resized
fmnist_test = x_test_resized

fmnist_train = fmnist_train.astype('float32') / 255.
fmnist_test = fmnist_test.astype('float32') / 255.

In [102]:
# Load the CIFAR-10 dataset
(cifar_train, _), (cifar_test, _) = cifar10.load_data()

# Convert images from 32x32x3 to 32x32x1 grayscale
x_train_gray = np.zeros((cifar_train.shape[0], cifar_train.shape[1], cifar_train.shape[2], 1), dtype=np.uint8)
x_test_gray = np.zeros((cifar_test.shape[0], cifar_test.shape[1], cifar_test.shape[2], 1), dtype=np.uint8)

for i in range(cifar_train.shape[0]):
    grayscale_image = cv2.cvtColor(cifar_train[i], cv2.COLOR_RGB2GRAY)
    x_train_gray[i, :, :, 0] = grayscale_image

for i in range(cifar_test.shape[0]):
    grayscale_image = cv2.cvtColor(cifar_test[i], cv2.COLOR_RGB2GRAY)
    x_test_gray[i, :, :, 0] = grayscale_image

# x_train_gray and x_test_gray now contain CIFAR-10 images in 32x32x1 grayscale format
# You can use these grayscale datasets for further processing
cifar_train = x_train_resized
cifar_test = x_test_resized

cifar_train = cifar_train.astype('float32') / 255.
cifar_test = cifar_test.astype('float32') / 255.

In [103]:
print(fmnist_train.shape, fmnist_test.shape)
print(mnist_train.shape, mnist_test.shape)
print(cifar_train.shape, cifar_test.shape)

(60000, 32, 32, 1) (10000, 32, 32, 1)
(60000, 32, 32, 1) (10000, 32, 32, 1)
(60000, 32, 32, 1) (10000, 32, 32, 1)


In [111]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

mnist_preds = predict(autoencoder, mnist_test, threshold)
fmnist_preds = predict(autoencoder, fmnist_test, threshold)
cifar_preds = predict(autoencoder, cifar_test, threshold)

#print(f'Mnist Accuracy: {compute_accuracy(mnist_preds)}')

In [114]:

print(f'Detection Accuracy on Mnist: {1- compute_accuracy(mnist_preds)}')
print(f'Detection accuracy on Fashion Mnist: {1- compute_accuracy(fmnist_preds)}')
print(f'Accuracy on Cifar10: {compute_accuracy(cifar_preds)}')

Detection Accuracy on Mnist: 0.0
Detection accuracy on Fashion Mnist: 0.0031999999999999806
Accuracy on Cifar10: 0.9968
